In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np

# shader constants
VERTEX_SHADER = """
#version 330

in vec3 position;
in vec3 color;
    
out vec3 newColor;

void main() {
    gl_Position = vec4(position, 1.0);
    newColor = color;
}
"""

FRAGMENT_SHADER = """
#version 330
    
in vec3 newColor;
    
out vec4 outColor;
    
void main() {
    outColor = vec4(newColor, 1.0f);
}
"""

# window size constants
WINDOW_HEIGHT = 600
WINDOW_WIDTH = 800
WINDOW_NAME = "eraumacasamuitoengraçada"
WINDOW_MONITOR = None
WINDOW_SHARE = None

# position length
POSITION_LENGTH = 3
COLOR_LEGTH = 3

# background color
BACKGROUND_COLOR = (1.0, 1.0, 1.0, 1.0)


def initiate_shader_things():
    # compiling the program and shaders
    shader = OpenGL.GL.shaders.compileProgram(
        OpenGL.GL.shaders.compileShader(VERTEX_SHADER, GL_VERTEX_SHADER),
        OpenGL.GL.shaders.compileShader(FRAGMENT_SHADER, GL_FRAGMENT_SHADER)
    )

    # create buffer object in GPU
    VBO = glGenBuffers(1)

    # binding the buffer
    glBindBuffer(GL_ARRAY_BUFFER, VBO)

    # getting the position from  shader
    position = glGetAttribLocation(shader, 'position')
    glVertexAttribPointer(
        position, 
        POSITION_LENGTH, 
        GL_FLOAT, 
        GL_FALSE, 
        4 * (POSITION_LENGTH + COLOR_LEGTH), 
        ctypes.c_void_p(0)
    )
    glEnableVertexAttribArray(position)

    # getting the color from  shader
    color = glGetAttribLocation(shader, 'color')
    glVertexAttribPointer(
        color, 
        COLOR_LEGTH, 
        GL_FLOAT, 
        GL_FALSE, 
        4 * (POSITION_LENGTH + COLOR_LEGTH), 
        ctypes.c_void_p(4 * POSITION_LENGTH)
    )
    glEnableVertexAttribArray(color)
    
    return shader


def initiate_things():
    # creates a window
    window = glfw.create_window(
        WINDOW_WIDTH, 
        WINDOW_HEIGHT, 
        WINDOW_NAME, 
        WINDOW_MONITOR, 
        WINDOW_SHARE
    )

    # check if the window was created succesfully
    if not window:
        glfw.terminate()
        return
    
    glfw.make_context_current(window)
    
    shader = initiate_shader_things()

    glUseProgram(shader)
    
    # background color
    glClearColor(*BACKGROUND_COLOR)
    
    return window


def draw_rectangles(rectangle) -> None:
    number_of_squares = (len(rectangle) // (POSITION_LENGTH + COLOR_LEGTH)) // 4
    
    glBufferData(GL_ARRAY_BUFFER, rectangle.nbytes, rectangle, GL_STATIC_DRAW)
    glDrawArraysInstanced(GL_QUADS, 0, 4*number_of_squares, number_of_squares)
    

def draw_triangles(triangle) -> None:
    number_of_triangles = (len(triangle) // (POSITION_LENGTH + COLOR_LEGTH)) // 3
    
    glBufferData(GL_ARRAY_BUFFER, triangle.nbytes, triangle, GL_STATIC_DRAW)
    glDrawArraysInstanced(GL_TRIANGLES, 0, 3*number_of_triangles, number_of_triangles)


def main():
    # check if is possible to initialize glfw
    if not glfw.init():
        return

    window = initiate_things()

    # defining the colors
    cinza = (129.0 / 255.0, 128.0 / 255.0, 129.0 / 255.0)
    azul_bebe = (176.0 / 255.0, 253.0 / 255.0, 254.0 / 255.0)
    marrom = (128.0 / 255.0, 89.0 / 255.0, 27.0 / 255.0)
    tijolo = (205.0 / 255.0, 88.0 / 255.0, 36.0 / 255.0)
    
    # defining the rectangles
    base = np.array(
        [   # Positions         #Colors
            ### BASE ###   
            -0.25, -0.5, 0.0,   *cinza,
            0.25, -0.5, 0.0,    *cinza,
            0.25, 0.0, 0.0,     *cinza,
            -0.25, 0.0, 0.0,    *cinza,
            ### PORTA ###
            -0.125, -0.5, 0.0,  *azul_bebe,
            0.0, -0.5, 0.0,     *azul_bebe,
            0.0, -0.25, 0.0,    *azul_bebe,
            -0.125, -0.25, 0.0, *azul_bebe,
            ### JANELA ###
            0.05, -0.3, 0.0,    *azul_bebe,
            0.2, -0.3, 0.0,     *azul_bebe,
            0.2, -0.15, 0.0,    *azul_bebe,
            0.05, -0.15, 0.0,   *azul_bebe,
            ### CHAMINÉ ###
            -0.2, 0.0, 0.0,     *marrom,
            -0.125, 0.0, 0.0,   *marrom,
            -0.125, 0.4, 0.0,   *marrom,
            -0.2, 0.4, 0.0,     *marrom 
        ], 
        dtype=np.float32
    )
    
    # defining the triangle
    telhado = np.array(
        [   #Postions           #Colors
            -0.25, 0.0, 0.0,    *tijolo,
            0.25, 0.0, 0.0,     *tijolo,
            0.0, 0.3, 0.0 ,     *tijolo
        ],
        dtype=np.float32
    )

    # loop to keep the window open and draw things
    while not glfw.window_should_close(window):
        glfw.poll_events()

        glClear(GL_COLOR_BUFFER_BIT)

        # draw rectangles
        draw_rectangles(base)
        
        # draw triangles
        draw_triangles(telhado)

        glfw.swap_buffers(window)

    glfw.terminate()


if __name__ == "__main__":
    main()